Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# Inference PyTorch Bert Model for High Performance in ONNX Runtime

In this tutorial, you'll be introduced to how to load a Bert model from PyTorch, convert it to ONNX, and inference it for high performance using ONNX Runtime. In the following sections, we are going to use the Bert model trained with Stanford Question Answering Dataset (SQuAD) dataset as an example. Bert SQuAD model is used in question answering scenarios, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.

## 0. Prerequisites ##
First we need a python environment with [PyTorch](https://pytorch.org/) and [OnnxRuntime](https://microsoft.github.io/onnxruntime/) installed before running this notebook.

#### GPU Environment Setup
In this example, we install [AnaConda](https://www.anaconda.com/distribution/) in a machine with GPU and open an AnaConda prompt window when it is done. Then run the following commands to create a conda environment. 

onnxruntime-gpu v1.1.2 requires installing [CUDA](https://developer.nvidia.com/cuda-downloads) 10.0 and [cuDNN](https://developer.nvidia.com/cudnn) 7.6, and add their bin directories to PATH environment variable.

```console
conda create -n gpu_env python=3.6
conda activate gpu_env
conda install pytorch torchvision cudatoolkit=10.1 -c pytorch
pip install onnxruntime-gpu
conda install jupyter
jupyter notebook
```

#### CPU Environment Setup
If your machines does not have GPU or want to test CPU inference. You can create a conda environment like the following:

```console
conda create -n cpu_env python=3.6
conda activate cpu_env
conda install pytorch torchvision cpuonly -c pytorch
pip install onnxruntime
conda install jupyter
jupyter notebook
```

The last command will launch Jupyter Notebook and we can open this notebook in browser to continue.

In [ ]:
import sys
!{sys.executable} -m pip install wget
!{sys.executable} -m pip install transformers
!{sys.executable} -m pip install psutil

## 1. Load Pretrained Bert model ##

We begin by downloading the data files and store them in the specified location. 

In [ ]:
import os

# Create a directory to store predict file
output_dir = "./pytorch_output"
cache_dir = "./pytorch_squad"
predict_file = os.path.join(cache_dir, "dev-v1.1.json")
# create cache dir
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)
    
# Download the file
predict_file_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json"
if not os.path.exists(predict_file):
    import wget
    print("Start downloading predict file.")
    wget.download(predict_file_url, predict_file)
    print("Predict file downloaded.")

Specify some model config variables.

In [ ]:
model_name_or_path = "bert-base-cased"
max_seq_length = 128
doc_stride = 128
max_query_length = 64
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Start to load model from pretrained. This step could take a few minutes. 

In [ ]:
# The following code is adapted from HuggingFace transformers
# https://github.com/huggingface/transformers/blob/master/examples/run_squad.py

from transformers import (BertConfig, BertForQuestionAnswering, BertTokenizer)

# Load pretrained model and tokenizer
config_class, model_class, tokenizer_class = (BertConfig, BertForQuestionAnswering, BertTokenizer)
config = config_class.from_pretrained(model_name_or_path, cache_dir=cache_dir)
tokenizer = tokenizer_class.from_pretrained(model_name_or_path, do_lower_case=True, cache_dir=cache_dir)
model = model_class.from_pretrained(model_name_or_path,
                                    from_tf=False,
                                    config=config,
                                    cache_dir=cache_dir)
# load some examples
from transformers.data.processors.squad import SquadV1Processor

processor = SquadV1Processor()
examples = processor.get_dev_examples(None, filename=predict_file)

from transformers import squad_convert_examples_to_features
features, dataset = squad_convert_examples_to_features( 
            examples=examples[:3], # convert only 3 examples for demo
            tokenizer=tokenizer,
            max_seq_length=max_seq_length,
            doc_stride=doc_stride,
            max_query_length=max_query_length,
            is_training=False,
            return_dataset='pt'
        )

## 2. Export the loaded model ##
Once the model is loaded, we can export the loaded PyTorch model to ONNX.

In [ ]:
output_model_path = './pytorch_squad/bert-base-cased-squad.onnx'    

# Get the first batch of data to run the model and export it to ONNX
batch = dataset[0]

# Set model to inference mode, which is required before exporting the model because some operators behave differently in 
# inference and training mode.
model.eval()
model.to(device)
inputs = {
    'input_ids':      batch[0].to(device).reshape(1, 128),                         # using batch size = 1 here. Adjust as needed.
    'attention_mask': batch[1].to(device).reshape(1, 128),
    'token_type_ids': batch[2].to(device).reshape(1, 128)
}
with torch.no_grad():
    symbolic_names = {0: 'batch_size', 1: 'max_seq_len'}
    torch.onnx.export(model,                                            # model being run
                      (inputs['input_ids'],                             # model input (or a tuple for multiple inputs)
                       inputs['attention_mask'], 
                       inputs['token_type_ids']), 
                      output_model_path,                                # where to save the model (can be a file or file-like object)
                      opset_version=11,                                 # the ONNX version to export the model to
                      do_constant_folding=True,                         # whether to execute constant folding for optimization
                      input_names=['input_ids',                         # the model's input names
                                   'input_mask', 
                                   'segment_ids'],
                      output_names=['start', 'end'],                    # the model's output names
                      dynamic_axes={'input_ids': symbolic_names,        # variable length axes
                                    'input_mask' : symbolic_names,
                                    'segment_ids' : symbolic_names,
                                    'start' : {0: 'batch_size'},
                                    'end' : {0: 'batch_size'}})
    print("Model exported at ", output_model_path)

## 3. PyTorch Inference ##
Use PyTorch to evaluate an example input for comparison purpose.

In [ ]:
import time

# Warm up with one run.
model(**inputs)

# Measure the latency.
with torch.no_grad():
    start = time.time()
    outputs = model(**inputs)
    end = time.time()
    print("PyTorch {} Inference time = {} ms".format(device.type, format((end - start) * 1000, '.2f')))

## 4. Inference the Exported Model with ONNX Runtime ##

To use onnxruntime-gpu, it is required to install CUDA 10.0 and CUDNN 7.6, and add their bin directories to PATH environment variable.

In [ ]:
if device.type == 'cuda':
    # Add path for CUDA 10.0 and CUDNN 7.6.
    os.environ["PATH"] = 'E:/NVidia/CUDA/v10.0/bin;' + os.environ["PATH"]

Now we are ready to inference the model with ONNX Runtime.

In [ ]:
import psutil
import onnxruntime
import numpy

device_name = 'cuda' if 'CUDAExecutionProvider' in onnxruntime.get_available_providers() else 'cpu'

sess_options = onnxruntime.SessionOptions()

# Optional: store the optimized graph and view it using Netron to verify that model is fully optimized.
sess_options.optimized_model_filepath = os.path.join(output_dir, "optimized_model_{}.onnx".format(device_name))

# The following settings enables OpenMP, which is required to get best performance for CPU inference of Bert models.
sess_options.intra_op_num_threads=1
os.environ["OMP_NUM_THREADS"] = str(psutil.cpu_count(logical=True))
os.environ["OMP_WAIT_POLICY"] = 'ACTIVE'

session = onnxruntime.InferenceSession(output_model_path, sess_options)

# Use contiguous array as input could improve performance.
ort_inputs = {'input_ids': numpy.ascontiguousarray(inputs['input_ids'].cpu().numpy()),
              'input_mask': numpy.ascontiguousarray(inputs['attention_mask'].cpu().numpy()),
              'segment_ids': numpy.ascontiguousarray(inputs['token_type_ids'].cpu().numpy())
}

# Warm up with one run.
session.run(None, ort_inputs)

# Measure the latency.
start = time.time()
results = session.run(None, ort_inputs)
end = time.time()
print("ONNX Runtime {} inference time: {} ms".format(device_name, format((end - start) * 1000, '.2f')))

In [ ]:
print("***** Verifying correctness *****")
for i in range(2):
    print('PyTorch and ONNX Runtime output {} are close:'.format(i), numpy.allclose(results[i], outputs[i].cpu(), rtol=1e-05, atol=1e-04))

### 4. More Information ###
For more accurate latency number, it is recommended to measure 1000 inputs and get the statistics. 

You can try our [Performance Test and Model Verification Tools](https://github.com/microsoft/onnxruntime/tree/tlwu/bert_test_tools/onnxruntime/python/tools/bert) on the optimized model.

Example commands:

```console
python compare_bert_results.py --baseline_model ./pytorch_squad/bert-base-cased-squad.onnx --optimized_model ./pytorch_output/optimized_model_cpu.onnx --batch_size 1 --sequence_length 128 --samples 100

python bert_perf_test.py --model ./pytorch_output/optimized_model_cpu.onnx --batch_size 1 --sequence_length 128 --samples 100 --test_times 10 --inclusive
```